In [ ]:
import MySQLdb
import serial
import numpy as np
import pickle
from sklearn import tree

 
ser = serial.Serial('/dev/ttyACM2',115200)
db = MySQLdb.connect(host="localhost",user="root",passwd="2580",db="solar")
curs=db.cursor()

list_pickle_path = 'model_v1.pkl'
list_unpickle = open(list_pickle_path, 'rb')

clf = pickle.load(list_unpickle)
 
while 1:
    text = ser.readline()
    if "@@@" in text :
        temp_in = ser.readline()
        humi = ser.readline()
        temp_out = ser.readline()
        gas = ser.readline()
        co2 = ser.readline()
        soil_humidity = ser.readline()
        light = ser.readline()
        static = ser.readline()
        
        data = np.array([[temp_in, co2, gyro_x, gyro_y, gyro_z]])
        prediction = clf.predict(data)

        if prediction == 0:
            safe = 'Safe'
        elif prediction == 1:
            safe = 'Fire'
        else:
            safe = 'Fallen'
        
        try:

            sql = """INSERT INTO field(temperature, co2, humidity,temp_in,gas,safe,soil_humidity,light,static) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""%(temp_out,co2,humi,temp_in,gas,safe,soil_humidity,light,static)
            curs.execute(sql)
            db.commit()
            print "End db commit"
 
        except:
            print "Error: db commit"
            db.rollback()
    else:
        print text
        print type(text)
        print ("@@@" in text)

